In [ ]:
!pip install tensorflow

In [10]:
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Dropout, RepeatVector, Embedding , TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.losses import sparse_categorical_crossentropy


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ehsanh123/NLP-Week6.git

Cloning into 'NLP-Week6'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 5.17 MiB | 19.32 MiB/s, done.


In [4]:
# path ='/content/drive/MyDrive/NLP/spa.txt'
path = '/content/NLP-Week6/spa.txt'
traslation_file = open(path ,"r",encoding="utf-8")
raw_data = traslation_file.read()
traslation_file.close()

In [5]:
from typing import Sequence
raw_data = raw_data.split('\n')
pairs = [Sequence.split('\t') for Sequence in raw_data]
pairs = pairs[1000:20000]

In [9]:
pairs[100]

['Tom cares.',
 'Tom se preocupa.',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #2203611 (CK) & #4557602 (swyter)']

In [6]:
def clean_sentence(sentece):
  lower_case = sentece.lower()
  string_punctuation = string.punctuation + "¿"
  clean_sentence = lower_case.translate(str.maketrans("","",string_punctuation))
  return clean_sentence

In [7]:
def tokenizer(sentece):
  text_tokenizer = Tokenizer()
  text_tokenizer.fit_on_texts(sentece)
  return text_tokenizer.texts_to_sequences(sentece), text_tokenizer

In [12]:
# Clean sentences
english_sentences = [clean_sentence(pair[1]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[0]) for pair in pairs]
# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenizer(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenizer(english_sentences)
print('Maximum length spanish sentence: {}'.format (len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format (len (max(eng_text_tokenized, key=len))))
# Check language length
spanish_vocab = len(spa_text_tokenizer. word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of () unique words".format (spanish_vocab))
print ("English vocabulary is of f) unique words".format (english_vocab))

Maximum length spanish sentence: 5
Maximum length english sentence: 9
Spanish vocabulary is of () unique words
English vocabulary is of f) unique words


In [13]:
max_spanish_len = int(len(max(spa_text_tokenized, key=len)))

max_english_len = int(len(max(eng_text_tokenized, key=len)))

spa_pad_sentence = pad_sequences (spa_text_tokenized, max_spanish_len, padding = "post")

eng_pad_sentence = pad_sequences (eng_text_tokenized, max_english_len, padding = "post")

#Reshape data

spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)

eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [14]:
input_sequence = Input(shape=(max_spanish_len,))

embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)

encoder = LSTM (64, return_sequences=False) (embedding)

r_vec = RepeatVector(max_english_len) (encoder)

decoder = LSTM (64, return_sequences=True, dropout=0.2) (r_vec)

logits = TimeDistributed (Dense(english_vocab)) (decoder)

In [15]:
enc_dec_model = Model(input_sequence, Activation('softmax') (logits))

# enc_dec_model.compile(loss=sparse_categorical_crossentropy,
# optimizer=Adam(1e-3), metrics=['accuracy'])

enc_dec_model.compile(loss=sparse_categorical_crossentropy,
optimizer=Adam(1e-3), metrics=['accuracy'])

enc_dec_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 5, 128)              │         475,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 9, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 9, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 9, 7423)             │         482,495 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 9, 7423)             │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,040,831 (3.97 MB)

 Trainable params: 1,040,831 (3.97 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_results = enc_dec_model.fit(spa_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)

Epoch 1/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.6410 - loss: 4.1233
Epoch 2/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6644 - loss: 2.4448
Epoch 3/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6672 - loss: 2.3327
Epoch 4/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6762 - loss: 2.2335
Epoch 5/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.6864 - loss: 2.1099
Epoch 6/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.6970 - loss: 1.9974
Epoch 7/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.7041 - loss: 1.9103
Epoch 8/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.7104 - loss: 1.8362
Epoch 9/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7176 - loss: 1.7587
Epoch 10/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.7231 - loss: 1.6937
Epoch 11/100
634/634 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.7292 - loss: 1.6305
Epoch 12/100
634/634 ━━

In [ ]:
# Example test Sentences
test_sentences = [
"How are you",
"Today is Friday",
"This is NLP"
]

# Clean and Tokenize Test Sentences
clean_test_sentences = [clean_sentence (sentence) for sentence in test_sentences]

#Tokenize Test Sentences

test_sequences_spa = spa_text_tokenizer.texts_to_sequences(clean_test_sentences)

test_pad_sequences_spa = pad_sequences(test_sequences_spa, max_spanish_len, padding-"post")

# Reshape Test Data
test_pad_sequences_spa = test_pad_sequences_spa.reshape(*test_pad_sequences_spa.shape, 1)

print(test_pad_sequences_spa)

#Predict Translations

predicted_sequences = enc_dec_model.predict(test_pad_sequences_spa)

#Convert Indices to Words

def sequences_to_text(sequences, tokenizer):

  word_index = tokenizer.word_index

  reverse_word_index = dict([(index, word) for (word, index) in word_index.items()])

  return ' '.join([reverse_word_index.get(i,'?') for i in sequences])

# Display Results

for i, sentence in enumerate(test_sentences):
  print("Input Sentence: (sentence)")
  print(f"Predicted Translation: (sequences_to_text(np.argmax(predicted_sequences[i], axis-1), eng_text_tokenizer))")
  print()